# Noisy channel spelling correction

## Resources

Norvig resources

1. [Spell checking blog post](http://norvig.com/spell-correct.html)
2. [count_1edit.txt: error counts file for 1 edit errors](https://norvig.com/ngrams/count_1edit.txt)


A couple of comments on the resources above.  

Norvig's Python spelling correcter is beautiful python code (like his [Sudoku solver](https://norvig.com/sudoku.html)), but it isn't actually a noisy channel model, because it doesn't use a probabilistic error model, although he tries to talk into you into thinking that it does.  However, it's designed so that you could turn it into a real noisy channel model quite easily, and you might try looking at the code and thinking about how to do that (but read the blog post first, so as to have a take on how to approach the code).
An extract from that code is used in this assignment.

The file `count_1edit.txt` is based obn data about misspellings (see Norvig's post) and contains counts of errors that are 1 edit away from the correct word. We use those counts for our error model.  We use `spell-errors.txt`, the source file from which `count_1edit.txt` was derived, to get the total unigraph and bigraph counts.


## Setup code

In [1]:
import nltk
from nltk.corpus import brown
import codecs
import re
import urllib

def open_file_or_url(path):
    if re.match(r'https?://', path):
        return urllib.request.urlopen(path)
    else:
        # Paralleling what urlopen does, we are going to open a bytestream (not a decoded string stream)
        # and read and decode it separately.
        return open(path, 'rb')
 
def get_letter_counts (path, encoding='latin1', linesep = r'\n', errors='strict'):
    """
    Build a single nltk.FreqDist instance with counts of both unigraphs
    and bigraphs for the words in `data_file`.
    """
    global line
    letters = []
    with open_file_or_url(path) as handle:
        bytesdata = handle.read()
        data = codecs.decode(bytesdata,encoding=encoding,errors=errors)
    for (lct,line) in enumerate(re.split(linesep,data)):
        line = line.strip()
        if line:
           #print(line)
           (right,wrong) = line.split(":")
           letters.append(right)
           letters.extend(ww.strip()  for ww in wrong.split(","))
    # One big long string of letters with spaces separating words
    letters = ' '.join(letters)
    fd = nltk.FreqDist(letters)
    blts = list(nltk.bigrams(letters))
    fd.update(blts)
    return (fd, letters)

You will need to execute the following download code each time you run the notebook if you are running in google colab.  If you run this on your local machine, that will install the `brown` corpus permanently on your machine and you will only have to run this cell once.

In [2]:
nltk.download('brown')

[nltk_data] Downloading package brown to /Users/gawron/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

Get the counts necessary for computing bigram model for the lower-cased Brown corpus.  This takes a littke while.

In [3]:
from nltk.corpus import brown
brown_lc_words = [wd.lower() for wd in brown.words()]
WORDS = set(brown_lc_words)
ugr_words =  nltk.FreqDist(brown_lc_words)
bigr_words =  nltk.FreqDist(nltk.bigrams(brown_lc_words))

In [4]:
big_voc = list(bigr_words.keys())

In [5]:
[(w1,w2) for (w1,w2) in big_voc if w2 == 'francisco']

[('san', 'francisco'), (',', 'francisco'), ('of', 'francisco')]

In [6]:
ugr_words['to']

26158

Get the data from the mispelling lists Norvig used to compile error counts.

Compute the necessary **unigraph** and **bigraph** counts for what's coming.  We'll call these kinds of information our **ngraph model**.

In [7]:
url = 'https://norvig.com/ngrams/spell-errors.txt'
letter_freqs, letters = get_letter_counts (url)

In [8]:
letter_freqs

FreqDist({' ': 47550, 'e': 46211, 'i': 29799, 'a': 29298, 'n': 27727, 't': 26161, 's': 24828, 'r': 24599, 'o': 21493, 'l': 18625, ...})

The `letterFreqs` dictionary contains both unigram letter frequencies and bigram letter frequencies.

In [9]:
letter_freqs['i']

29799

In [10]:
letter_freqs['o','r']

2857

## Load Norvig's error data

Executing the function defined in the next cell requires that there be a copy of the file `count_1edit.txt` in the same directory as this notebook. I have tried to facilitate this by supplying you with a zip file that includes the error data file.  Reading the file in to Python only worked for me if I specified the Python `'latin1'` codec, 
but once I did that, I  was able to use the identical file supplied on
[Norvig's NLP code and data page](https://norvig.com/ngrams/).  

If you have trouble finding the error file on your computer, just edit the cell that calls `make_err_dict` to
use a full path name:

```
make_error_dict(<full_path_string_to count_1edit.txt>)
```

In [11]:
 
def make_err_dict (path, encoding='latin1',errors='strict',linesep = r'\n'):
    """
    To get past some funky unicode characters, do errors = 'ignore'
    
    This will lose the troublesome characters; it  will not perform a reasonable
    approximation, such as "e"  for "e with an accent mark". 
    """
    global error_cts
    error_cts = dict()
    with open_file_or_url(path) as handle:
        bytesdata = handle.read()
        data = codecs.decode(bytesdata,encoding=encoding,errors=errors)
    for (lct,line) in enumerate(re.split(linesep,data)):
        process_line (line, error_cts, lct)
    return error_cts

def process_line (line, error_cts,lct):
     line = line.strip()
     if line:
        (err, ct) = line.split('\t')
        (tgt, src) = err.split('|')
        if tgt and src:
           error_cts[tgt,src] = int(ct)
        elif tgt:
            error_cts[tgt,'#'] = int(ct)
        elif src:
            error_cts['#', src] = int(ct)
        else:
            print(f'***Warning*** Bad line {lct}: {line}')

The `make_err_dict` function loads Norvig's data on error counts and makes a dictionary of it.  We'll call that our **error counts data**.

In [12]:
# Error data on Peter Norvig's website
url = 'https://norvig.com/ngrams/count_1edit.txt'
err_dict = make_err_dict(url)
# OR Downloaded as a file on your local machine
#fn = 'count_1edit.txt'
#err_dict = make_err_dict(fn)

***Warning*** Bad line 415: |	19


In [13]:
#count('e'|'i'): Sub e for i
# This shd return 917
err_dict['e','i']

917

In [14]:
#count('es'|'e'): Insert s after e
#This should return 136
err_dict['es','e']

136

Some errors are not that frequent:

In [ ]:
err_dict['eh','he']

6

Some errors we simply have no data for:

In [ ]:
err_dict['he','eh']

KeyError: ('he', 'eh')

To compute an error probability, we need the ngram model (`ugr_letter_freqs` and `bigr_letter_freqs`) and the error counts data (`err_dict`), both loaded in cells above this one.

Using the probability formula for an insertion on Slide 24, we get:

In [ ]:
#P(es|e)
err_dict["es","e"]/letter_freqs['e']

0.002943022224145766

From Peter Norvig's `spell.py`, the function `candidates` defined in the next cell is an amended version of Norvig's Python function.  It generates candidate corrections one edit distance away from a misspelling.

In [ ]:
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or  [word])


def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All 'words' that are one edit operation away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


In [ ]:
candidates('potch')

{'notch', 'patch', 'pitch', 'poach', 'porch', 'pouch'}

In [ ]:
candidates('pooch')

{'cooch', 'hooch', 'poach', 'porch', 'pouch'}

Our model doesn't know the word "pooch" (it just didn't show up in Brown).  If you give `candidates` a word that **is** in Brown, it only returns one candidate:

As a result, the spelling correcter we're building won't handle mispellings that are real words ("thew" for "the").

## The assignment

Here is a sentence with a mispelling.  All punctuation has been removed because your error model doesn't include
punctuation errors.

```
I found John on the back potch of his house stroking a big brown dog
```

1.  Use `err_dict`, and `letter_freqs` to compute your error probabilities (P(x | w))
    using the formulas on the slides.  You can safely assume any misspelling in the sentence above is 1 edit
    distance from the intended word, and that's what the candidate-generating code below does.

2.  Find the candidate correct words using Norvig's `candidates` function defined below.  For instance,
    given the mispelling "thorw", `candidates` would find
```
>>> candidates('thorw')
{'thor', 'thorn', 'thorp', 'throw'}
```
    The inclusion of 'thor' and 'thorp' is probably because they are proper names and we have lower-cased the corpus.     Don't worry about this.  For this assignment, just consider every candidate `candidates` returns.

3.  For our language model we will compute the probability of each candidate  word w in context:
    $\mbox{P}_{con}(\mbox{w})$.  To compute that, use the bigram language model defined by the lower-cased 
    Brown corpus we loaded above to compute $\mbox{p}_{con}(\mbox{w})$ (the probability of candidate word w in context). You do not     have to consider only the previous word when you use a bigram model.  You can take both the preceding word $\mbox{prev}$
    and the following word $\mbox{foll}$ into account by computing the probability of 
    the 3-word sequence consisting of prev, w, foll; so  $mbox{p}_{con}(\mbox{w})$, the provbability
    of candidate word w in the context of words prev and foll, is:
    
    $$
    \mbox{p}_{con}(\mbox{w}) = p(\mbox{prev}) * p(\mbox{w} \mid \mbox{prev}) * P(\mbox{foll} \mid \mbox{c})
    $$
    
    So for example if our given sentence included the three word sequence "boys thorw the" and we wanted to 
    consider to correct it to "boys throw the", the candidate word would be throw and we would have:

    $$
    \mbox{p}_{con}(\mbox{throw}) = \mbox{p}(\mbox{boys}) * \mbox{p}(\mbox{throw} \mid \mbox{boys}) * 
       \mbox{p}(\mbox{the} \mid \mbox{throw})
    $$
    
    You may use an unsmoothed MLE model as your bigram model.

4.  For each candidate word w, use the noisy channel model to score all the candidates.

$$
p(x \mid \mbox{w}) * p_{con}(\mbox{w})
$$

5.  State what the best correction the model has chosen is.
6.  To turn in.  An edited version of this Python notebook that you will save (using the pull down menu "File"
    and choosing "Save and checkpoint") and rename to "[your_first_name]_[your_last_name]_noisy_channel_nitebook.ipynb", for
    example "joe_smith_spell.ipynb" (using the "File" menu and choosing "Rename").  You can create new cells
    by investigating the "Help" Menu and looking at "Keyboard Shortcuts".  Look for "Change cell to markdown" to 
    cells contain text, like this one.